In [31]:
import json
import numpy as np
import matplotlib.pyplot as plt

In [32]:
with open("manual_data.json", "r") as f:
    manual_def = json.load(f)

In [33]:
tile = [(0,0)]*25
for item in manual_def["treasureIslandTiles"]:
    x = item['coordinates']['x']
    y = item['coordinates']['y']
    tile[(x-26)*5 + (y-1)] = (item['multiplier'],item['hunters'])

In [34]:
n_strat_1 = len(tile)
n_strat_2 = len(tile)*(len(tile)-1)/2
n_strat_3 = len(tile)*(len(tile)-1)*(len(tile)-2)/6
total_strat = n_strat_1 + n_strat_2 + n_strat_3

In [35]:
strat_1 = []
strat_2 = []
strat_3 = []
for i in range(len(tile)):
    strat_1.append((i,))
    for j in range(i+1, len(tile)):
        strat_2.append((i,j))
        for k in range(j+1, len(tile)):
            strat_3.append((i,j,k))
            
p_strat_1 = [1/total_strat]*len(strat_1)
p_strat_2 = [1/total_strat]*len(strat_2)
p_strat_3 = [1/total_strat]*len(strat_3)

def tile_reward(tile, p_tiles, idx):
    return 7.5*tile[idx][0]/(tile[idx][1] + p_tiles[idx]*100)

In [36]:
for i in range(10000):
    # Calc other strats
    p_tiles = np.zeros(25)
    for strat,p_strat in zip(strat_1, p_strat_1):
        p_tiles[strat[0]] += p_strat

    for strat,p_strat in zip(strat_2, p_strat_2):
        p_tiles[strat[0]] += p_strat
        p_tiles[strat[1]] += p_strat

    for strat,p_strat in zip(strat_3, p_strat_3):
        p_tiles[strat[0]] += p_strat
        p_tiles[strat[1]] += p_strat
        p_tiles[strat[2]] += p_strat

    p_tiles /= np.sum(p_tiles)

    reward_1 = []
    for strat in strat_1:
        reward_1.append(tile_reward(tile, p_tiles, strat[0]))

    reward_2 = []
    for strat in strat_2:
        reward_2.append(tile_reward(tile, p_tiles, strat[0]) + tile_reward(tile, p_tiles, strat[1])-25)

    reward_3 = []
    for strat in strat_3:
        reward_3.append(tile_reward(tile, p_tiles, strat[0]) + tile_reward(tile, p_tiles, strat[1]) + tile_reward(tile, p_tiles, strat[2]) -75)

    reward_1 = np.array(reward_1)*0.001
    reward_2 = np.array(reward_2)*0.001
    reward_3 = np.array(reward_3)*0.001

    sum1 = np.inner(p_strat_1,np.exp(reward_1))
    sum2 = np.inner(p_strat_2,np.exp(reward_2))
    sum3 = np.inner(p_strat_3,np.exp(reward_3))

    norm_fact = sum1+sum2+sum3
    
    p_strat_1 = p_strat_1*np.exp(reward_1)/norm_fact
    p_strat_2 = p_strat_2*np.exp(reward_2)/norm_fact
    p_strat_3 = p_strat_3*np.exp(reward_3)/norm_fact
    

In [38]:
p_strat = np.concatenate([p_strat_1,p_strat_2,p_strat_3])

In [54]:
idx_argsort = np.argsort(-p_strat)

In [62]:
strat = strat_1 + strat_2 + strat_3
reward = 1000*np.concatenate([reward_1, reward_2, reward_3])

In [82]:
for i in range(2024):
    print(list(map(lambda x:((x//5 + 26),(1 +x % 5)), strat[idx_argsort[i]])), reward[idx_argsort[i]], p_strat[idx_argsort[i]])

[(27, 3), (28, 2), (28, 4)] 86.25000000000298 0.0029163559489965313
[(27, 3), (27, 4), (28, 2)] 86.25000000000301 0.002777923724483168
[(27, 2), (27, 3), (28, 2)] 86.25000000000301 0.002709493314025173
[(27, 3), (27, 4), (28, 4)] 86.25000000000298 0.002650169165878632
[(26, 3), (27, 3), (28, 2)] 86.25000000000298 0.002585362431962529
[(27, 2), (27, 3), (28, 4)] 86.25000000000298 0.0025848858169494507
[(27, 3), (28, 2), (29, 2)] 86.25000000000298 0.0025741614494164565
[(27, 4), (28, 2), (28, 4)] 86.25000000000301 0.00253188446603965
[(27, 3), (28, 2), (29, 4)] 86.25000000000298 0.002507243602150945
[(27, 3), (28, 2), (28, 3)] 86.25000000000301 0.002473968206423345
[(27, 2), (28, 2), (28, 4)] 86.25000000000301 0.0024695149014197166
[(26, 3), (27, 3), (28, 4)] 86.25000000000298 0.0024664636179248657
[(27, 2), (27, 3), (27, 4)] 86.25000000000298 0.002462187662124884
[(27, 3), (28, 4), (29, 2)] 86.25000000000298 0.002455777759882925
[(27, 3), (28, 4), (29, 4)] 86.25000000000298 0.0023919374

AttributeError: 'numpy.ndarray' object has no attribute 'list'